In [3]:
import ipywidgets as widgets

In [4]:
def f(o):
    print('hi')

In [5]:
w = widgets.Button(description='Click me')


In [6]:
w

Button(description='Click me', style=ButtonStyle())

In [7]:
w.on_click(f)

## Creating your own callback

In [8]:
from time import sleep


In [9]:
def slow_calculation(cb:None): 
    res = 0
    for i in range(5): 
        res +=i*i
        sleep(1)
        if cb: 
            cb(i)
    return res

In [10]:
slow_calculation(None)

30

In [11]:
def show_progress(epoch): 
    print(f"Awesome! we've finished epoch {epoch}!")

In [12]:
slow_calculation(show_progress)

Awesome! we've finished epoch 0!
Awesome! we've finished epoch 1!
Awesome! we've finished epoch 2!
Awesome! we've finished epoch 3!
Awesome! we've finished epoch 4!


30

## Lambdas and partials

In [13]:
slow_calculation(lambda o: print(f"Awesome! We've finished epoch {o}!"))

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

In [14]:
def show_progress(exclamation,epoch): 
    print(f"{exclamation}! We've finished epoch {epoch}!")

In [15]:
slow_calculation(lambda o: show_progress("OK I guess", o))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [16]:
def make_show_progress(exclamation):
    _inner = lambda epoch: print(f"{exclamation}! We finished epoch {epoch}")
    return _inner

In [17]:
slow_calculation(make_show_progress("Nice!"))

Nice!! We finished epoch 0
Nice!! We finished epoch 1
Nice!! We finished epoch 2
Nice!! We finished epoch 3
Nice!! We finished epoch 4


30

In [18]:
def make_show_progress(exclamation): 
    def _inner(epoch): 
        print(f"{exclamation}! We ve finished {epoch}!")
    return _inner

In [19]:
slow_calculation(make_show_progress('Nice'))

Nice! We ve finished 0!
Nice! We ve finished 1!
Nice! We ve finished 2!
Nice! We ve finished 3!
Nice! We ve finished 4!


30

In [20]:
from functools import partial 

In [21]:
slow_calculation(partial(show_progress, 'amazine'))

amazine! We've finished epoch 0!
amazine! We've finished epoch 1!
amazine! We've finished epoch 2!
amazine! We've finished epoch 3!
amazine! We've finished epoch 4!


30

## QUESTION 

What is the advantagen of using partial and what dose it really do so greate? 

## Callbacks and callabale classes

In [22]:
class ProgressShowingCallback(): 
    def __init__(self,exclamation): 
        self.exclamation = exclamation
    def __call__(self,epoch): 
        print(f"{self.exclamation}! We've finished epoch {epoch}")
        

In [23]:
cb=ProgressShowingCallback("Just super")

In [24]:
slow_calculation(cb)

Just super! We've finished epoch 0
Just super! We've finished epoch 1
Just super! We've finished epoch 2
Just super! We've finished epoch 3
Just super! We've finished epoch 4


30

## Multiple callback funcs; *args and **kwargs 

In [25]:
def f(*args,**kwargs): 
    print(f"args:{args}; kwargs:{kwargs}") 

In [26]:
f(3,3,'a','a',item1=None, item2=None,item3=None)

args:(3, 3, 'a', 'a'); kwargs:{'item1': None, 'item2': None, 'item3': None}


args means arguments,kwargs means keyword arguments. By using these variblaes we can send in arguments and key arguments without unfolding them and we can sedn stuff deep down in our library. 

In [27]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, val=res)
    return res

In [28]:
class PrintStepCallback(): 
    def __init__(self): 
        pass
    def before_calc(self,*args,**kwargs): 
        print(f"About to start")
    def after_calc(self,*args,**kwargs): 
        print(f"Done step")

In [29]:
slow_calculation(PrintStepCallback())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In [30]:
class PrintStepCallback(): 
    def __init__(self): 
        pass
    def before_calc(self,epoch,**kwargs):
        print(f"About to start {epoch}")
    def after_calc(self,epoch,val,**kwatgs): 
        print(f"After {epoch}: {val}")
    #def after_calc(self,epoch,val,**kwargs):
    #    print(f"After {epoch}: {val}")


In [31]:
slow_calculation(PrintStepCallback())

About to start 0
After 0: 0
About to start 1
After 1: 1
About to start 2
After 2: 5
About to start 3
After 3: 14
About to start 4
After 4: 30


30

### Modifying the behevaior

In [32]:
def slow_calculation(cb:None): 
    res=0
    for i in range(5): 
        if cb and hasattr(cb,'before_calc'): 
            cb.before_calc(i,res)
        res+=i*i     
        sleep(1)
        if cb and hasattr(cb,'after_calc'): 
            if cb.after_calc(i,res):
                print("stopping early")
                break
    return res
        
        

In [33]:
class PrintAfterCallback():
    def after_calc (self, epoch, val):
        print(f"After {epoch}: {val}")
        if val>10: return True


In [34]:
slow_calculation(PrintAfterCallback())

After 0: 0
After 1: 1
After 2: 5
After 3: 14
stopping early


14

In [35]:
class SlowCalculator(): 
    def __init__(self,cb=None): 
        self.cb=cb
        self.res=0
    def callback(self,cb_name,*args):
        if not self.cb: 
            return 
        cb=getattr(self.cb,cb_name,None)
        if cb: 
            return cb(self,*args)
        
    def calc(self): 
        for i in range(5): 
            self.callback('before_calc',i)
            self.res+=i*i
            sleep(1)
            if self.callback('after_calc',i):
                print("stopping early")
                break

In [36]:
class ModifyingCallback(): 
    def after_calc(self,calc,epoch): 
        print(f"After {epoch}: {calc.res}")
        if calc.res>10: return True
        if calc.res<3: calc.res = calc.res*2
    def before_calc(self,*args): 
        print('godisss')

In [37]:
calculator = SlowCalculator(ModifyingCallback())

In [38]:
calculator.calc()
calculator.res

godisss
After 0: 0
godisss
After 1: 1
godisss
After 2: 6
godisss
After 3: 15
stopping early


15

## __dunder__ thingies¶


In [39]:
class SloppyAdder():
    def __init__(self,o): 
        self.o=o
    def __add__(self,b): 
        return SloppyAdder(self.o + b.o + 0.01)
    def __repr__(self): 
        return str(self.o)

In [40]:
a = SloppyAdder(1)
b = SloppyAdder(2)
a+b

3.01

reading upp on dunder methods https://docs.python.org/3/reference/datamodel.html#object.__init__
medium article https://blog.rmotr.com/python-magic-methods-and-getattr-75cf896b3f88

* `__getattr__` But using the __getattr__ magic method, we can intercept that inexistent attribute lookup and do something so it doesn’t fail. Read more on the link above
* `__getattribute__` __getattribute__ is similar to __getattr__, with the important difference that __getattribute__ will intercept EVERY attribute lookup, doesn’t matter if the attribute exists or not.
* `__setattr__` Called when an attribute assignment is attempted. This is called instead of the normal mechanism (i.e. store the value in the instance dictionary). name is the attribute name, value is the value to be assigned to it.
* `__del__` Called when the instance is about to be destroyed. This is also called a finalizer or (improperly) a destructor. If a base class has a __del__() method, the derived class’s __del__() method, if any, must explicitly call it to ensure proper deletion of the base class part of the instance.
* `__string__` that provides a “string representation” of your object
* `__init__` that serves as the object initializer (sometimes incorrectly referred to as constructor)
* `__add__` that allows you to “overload” the + operator.
* `__new__` Called to create a new instance of class cls. __new__() is a static method (special-cased so you need not declare it as such) that takes the class of which an instance was requested as its first argument. The remaining arguments are those passed to the object constructor expression (the call to the class). The return value of __new__() should be the new object instance (usually an instance of cls).
* `__enter__` Enter the runtime context related to this object. The with statement will bind this method’s return value to the target(s) specified in the as clause of the statement, if any.
* `__exit__` Exit the runtime context related to this object. The parameters describe the exception that caused the context to be exited. If the context was exited without an exception, all three arguments will be None.
* `__len__` Called to implement the built-in function len(). Should return the length of the object, an integer >= 0. Also, an object that doesn’t define a __bool__() method and whose __len__() method returns zero is considered to be false in a Boolean context.
* `__repr__` Called by the repr() built-in function to compute the “official” string representation of an object. If at all possible, this should look like a valid Python expression that could be used to recreate an object with the same value (given an appropriate environment). If this is not possible, a string of the form <...some useful description...> should be returned. The return value must be a string object. If a class defines __repr__() but not __str__(), then __repr__() is also used when an “informal” string representation of instances of that class is required.
* `__str__` Called by str(object) and the built-in functions format() and print() to compute the “informal” or nicely printable string representation of an object. The return value must be a string object. This method differs from object.__repr__() in that there is no expectation that __str__() return a valid Python expression: a more convenient or concise representation can be used. The default implementation defined by the built-in type object calls object.__repr__().

## Statisics and Variance

In [41]:
import torch
tensor = torch.tensor([1.,2,3,3,4,5,6,])

In [42]:
m = tensor.mean()
m

tensor(3.4286)

In [43]:
(tensor-m).mean()

tensor(-6.8120e-08)

In [44]:
(tensor-m).pow(2).mean()

tensor(2.5306)

In [45]:
(tensor-m).abs().mean()

tensor(1.3469)

In [46]:
(tensor-m).pow(2).mean().sqrt()


tensor(1.5908)

## Covariance and correlations

Here is the defenition of coveriance from wikpedia

$$\operatorname{cov}(X,Y) = \operatorname{E}{\big[(X - \operatorname{E}[X])(Y - \operatorname{E}[Y])\big]}$$

# Question 

Dose hard brakets here mean 
 ```
.mean()
```


In [51]:
# `u` is twice `t`, plus a bit of randomness
u = tensor*2
u *= torch.randn_like(tensor)/10+0.95

In [52]:
coveriance = (tensor-tensor.mean())*(u-u.mean()) 
coveriance


tensor([11.8039,  4.4845,  0.5085,  0.2623,  0.5202,  4.4299, 15.6052])

In [54]:
v = torch.randn_like(tensor)


In [55]:
v

tensor([ 1.1149,  0.0640, -0.8992, -0.0901,  1.5662, -1.2121,  1.7968])

In [57]:
((tensor-tensor.mean())*(v-v.mean())).mean()


tensor(0.1765)

In [59]:
cov = (tensor*v).mean() - tensor.mean()*v.mean(); cov


tensor(0.1765)

In [60]:
# Correlation
cov / (tensor.std() * v.std())


tensor(0.0866)